# Earthquake Magnitude Classification
A machine learning model to classify earthquake magnitude levels based on Latitude, Longitude, and Depth.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
def load_data(path):
    df = pd.read_csv(path)
    df = df.dropna()
    return df

In [ ]:
def label_magnitude(df):
    def classify(mag):
        if mag < 4.0:
            return 'Low'
        elif 4.0 <= mag < 6.0:
            return 'Moderate'
        else:
            return 'High'
    df['Magnitude_Class'] = df['Magnitude'].apply(classify)
    return df

In [ ]:
def preprocess_data(df):
    X = df[['Latitude', 'Longitude', 'Depth']]
    y = df['Magnitude_Class'].map({'Low': 0, 'Moderate': 1, 'High': 2})
    return train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

In [ ]:
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Low', 'Moderate', 'High'])
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nAccuracy: {acc:.4f}\n")
    print("Classification Report:\n", report)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Low', 'Moderate', 'High'],
                yticklabels=['Low', 'Moderate', 'High'])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig("confusion_matrix.png")
    plt.close()

In [ ]:
def save_artifacts(model, scaler):
    joblib.dump(model, 'earthquake_model.pkl')
    joblib.dump(scaler, 'scaler.pkl')

In [ ]:
data_path = 'earthquake_data.csv'
df = load_data(data_path)
df = label_magnitude(df)
X_train, X_test, y_train, y_test = preprocess_data(df)
X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)
model = train_model(X_train_scaled, y_train)
evaluate_model(model, X_test_scaled, y_test)
save_artifacts(model, scaler)